<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

## Interactive Python Data Science Dashboards 

### Bootstrap and Machine Learning Bagging with Custom Plots

#### Michael Pyrcz, Professor, The University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

This is a tutorial for / demonstration of **Bootstrap and Machine Learning Bagging with Custom Plots**. 

**YouTube Lecture**: check out my lectures on:

* [Bootstrap](https://youtu.be/wCgdoImlLY0?si=lpTWz2H7QTdxHBy9). 
* [Ensemble Tree Methods](https://www.youtube.com/watch?v=m5_wk310fho&list=PLG19vXLQHvSC2ZKFIkgVpI9fCjkN38kwf&index=39)

For your convenience here's a summary of salient points.

#### Bootstrap

Uncertainty in the sample statistics
* one source of uncertainty is the paucity of data.
* do 200 or even less wells provide a precise (and accurate estimate) of the mean? standard deviation? skew? P13?

Would it be useful to know the uncertainty in these statistics due to limited sampling?
* what is the impact of uncertainty in the mean porosity e.g. 20%+/-2%?

**Bootstrap** is a method to assess the uncertainty in a sample statistic by repeated random sampling with replacement.

Assumptions
* sufficient, representative sampling, identical, idependent samples

Limitations
1. assumes the samples are representative 
2. assumes stationarity
3. only accounts for uncertainty due to too few samples, e.g. no uncertainty due to changes away from data
4. does not account for boundary of area of interest 
5. assumes the samples are independent
6. does not account for other local information sources

The Bootstrap Approach (Efron, 1982)

Statistical resampling procedure to calculate uncertainty in a calculated statistic from the data itself.
* Does this work?  Prove it to yourself, for uncertainty in the mean solution is standard error: 

\begin{equation}
\sigma^2_\overline{x} = \frac{\sigma^2_s}{n}
\end{equation}

Extremely powerful - could calculate uncertainty in any statistic!  e.g. P13, skew etc.
* Would not be possible access general uncertainty in any statistic without bootstrap.
* Advanced forms account for spatial information and sampling strategy (game theory and Journel’s spatial bootstrap (1993).

Steps: 

1. assemble a sample set, must be representative, reasonable to assume independence between samples

2. optional: build a cumulative distribution function (CDF)
    * may account for declustering weights, tail extrapolation
    * could use analogous data to support

3. For $\ell = 1, \ldots, L$ realizations, do the following:

    * For $i = \alpha, \ldots, n$ data, do the following:

        * Draw a random sample with replacement from the sample set or Monte Carlo simulate from the CDF (if available). 

6. Calculate a realization of the sammary statistic of interest from the $n$ samples, e.g. $m^\ell$, $\sigma^2_{\ell}$. Return to 3 for another realization.

7. Compile and summarize the $L$ realizations of the statistic of interest.

#### Machine Learning Bagging

The fundamental idea is to use multiple data sets to built an ensemble of prediction models to calculate and aggregate over multiple predictions to reduce model variance.

* But, to build the ensemble of models we need multiple training datasets. This is typically not available.

* the solution is to **bootstrap** the entire dataset to build multiple bootstrap realizations of training data, $X_1^b,...,X_m^b$

* a deep decision tree is fit to each realization of the training data, $\hat{f}^b(X_1^b,...,X_m^b)$

* a prediction estimate is calculated by each tree in the ensemble $Y^b =\hat{f}^b(X_1^b,...,X_m^b)$

* for **regression** the ensemble prediction is the average of the prediction from each member of the ensemble, $Y = \frac{1}{B} \sum_{b=1}^{B} Y^b$

* for **classification** the ensemble prediction is the majority-rule of the ensemble classifications, $Y = argmax_k(Y^b_k)$

These are a very powerful method. Let's demonstrate them both with useful custom plots in interactive Python dashboards.

#### Load the required libraries

The following code loads the required libraries. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt                           # plotting
import statsmodels.api as sm                              # make PDFs
import numpy as np                                        # working with arrays
import pandas as pd                                       # working with DataFrames
from sklearn.linear_model import LinearRegression
from ipywidgets import interactive                        # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
import random                                             # random drawing / bootstrap realizations of the data

If you get a package import error, you may have to first install some of these packages. This can usually be accomplished by opening up a command window on Windows and then typing 'python -m pip install [package-name]'. More assistance is available with the respective package docs.  

#### Declare Functions

I just added a convenience functions to:

1. custom bootstrap plot
2. custom machine learning bagging plot

For these plots we make our own histogram and scatter plot with linear model plots to concisely visualize the entire workflow.

In [2]:
def custom_hist(data,ax,xpmin,xpmax,ypmin,ypmax,xmin,xmax,ymin,ymax,nbin,nybin,xlabel,ylabel,title,tsize,labelpad,
                plot_avg,plot_PDF):
    
    plt.plot([xpmin,xpmax],[ypmin,ypmin],color='black'); plt.plot([xpmin,xpmin],[ypmin,ypmax],color='black')
    xrange = xmax - xmin; yrange = ymax - ymin
    xprange = xpmax - xpmin; yprange = ypmax - ypmin
    xhalf = (xmax-xmin)/(nbin-1)/2.0; xsize = xhalf*2.0
    xphalf = xhalf*xprange/xrange
    xpsize = xsize*xprange/xrange
    xbins = np.linspace(xmin,xmax,nbin)
    ybins = np.linspace(0.0,ymax,nybin)
    xcents = np.linspace(xmin + xhalf,xmax-xhalf,nbin-1)
    
    xvalues = np.linspace(xmin,xmax,100)
    for ibin, xbin in enumerate(xbins):
        xx = ((xbin-xmin)/xrange*xprange)+xpmin
        plt.plot([xx,xx],[ypmin,ypmax],color='grey',lw=0.2,zorder=1)  
        if ibin % 2 == 0:
            plt.plot([xx,xx],[ypmin-yprange*0.1,ypmin],color='black',zorder=3)
            plt.annotate(np.round(xbin,1),(xx,ypmin-yprange*0.24),ha='center',size=tsize)
        else: 
            plt.plot([xx,xx],[ypmin-yprange*0.05,ypmin],color='black',zorder=3)
            plt.annotate(np.round(xbin,1),(xx,ypmin-yprange*0.15),ha='center',size=tsize)
        
    for ybin in ybins:
        yy = (ybin)*yrange/ymax*yprange/yrange+ypmin
        plt.plot([xpmin-xprange*0.04,xpmin],[yy,yy],color='black',zorder=3)
        plt.plot([xpmin,xpmax],[yy,yy],color='grey',lw=0.2,zorder=1)
        plt.annotate(np.round(ybin,1),(xpmin-xprange*0.05,yy),ha='right',size=tsize)
    
    for idata in range(0,len(data)):
        xx = ((data[idata]-xmin)/xrange*xprange)+xpmin 
        plt.scatter(xx,ypmin,color='red',edgecolor='black',s=20,alpha=0.4,zorder=100)
    
    histboot = np.histogram(data, bins=xbins, weights=None)[0]
    average = np.average(data)
    
    for ibin, prop in enumerate(histboot):
        xx = ((xcents[ibin]-xmin)/xrange*xprange)+xpmin
        yy = histboot[ibin]*yrange/ymax*yprange/yrange
        ax.add_patch(plt.Rectangle((xx-xphalf,ypmin), xpsize, yy, lw=1, fc = 'darkorange',color='black', ))
        xavg = ((average-xmin)/xrange*xprange)+xpmin
    
    if plot_avg:
        xx = ((np.average(data)-xmin)/xrange*xprange)+xpmin
        plt.plot([xx,xx],[ypmin,ypmax],color='red',lw=2,ls='--')
        
    if plot_PDF:
        PDFModel = sm.nonparametric.KDEUnivariate(data).fit()
        yPDF = PDFModel.evaluate(xvalues)*len(data)*xsize
        xxvalues = ((xvalues-xmin)/xrange*xprange)+xpmin; yyPDF = ((yPDF-ymin)/yrange*yprange)+ypmin
        plt.plot(xxvalues,yyPDF,color='black',zorder=200)
    
    plt.annotate(title,(xpmin+xprange*0.5,ypmax+yprange*0.08),ha='center') 
    plt.annotate(xlabel,(xpmin+xprange*0.5,ypmin-labelpad*1.5),ha='center') 
    plt.annotate(ylabel,(xpmin-labelpad*0.6,ypmin+yprange*0.5),va='center',rotation = 90.0) 
    
def custom_scatter(X,y,ax,xpmin,xpmax,ypmin,ypmax,xmin,xmax,ymin,ymax,nbin,nybin,xlabel,ylabel,title,tsize,labelpad,
                   add_model,show_pred,x_input):

    plt.plot([xpmin,xpmax],[ypmin,ypmin],color='black'); plt.plot([xpmin,xpmin],[ypmin,ypmax],color='black')
    xrange = xmax - xmin; yrange = ymax - ymin
    xprange = xpmax - xpmin; yprange = ypmax - ypmin
    xbins = np.linspace(xmin,xmax,nbin)
    ybins = np.linspace(ymin,ymax,nybin)
    
    for ibin, xbin in enumerate(xbins):
        xx = ((xbin-xmin)/xrange*xprange)+xpmin
        plt.plot([xx,xx],[ypmin,ypmax],color='grey',lw=0.2,zorder=1)  
        if ibin % 2 == 0:
            plt.plot([xx,xx],[ypmin-yprange*0.1,ypmin],color='black',zorder=3)
            plt.annotate(np.round(xbin,1),(xx,ypmin-yprange*0.24),ha='center',size=tsize)
        else: 
            plt.plot([xx,xx],[ypmin-yprange*0.05,ypmin],color='black',zorder=3)
            plt.annotate(np.round(xbin,1),(xx,ypmin-yprange*0.15),ha='center',size=tsize)
    
    for ibin, ybin in enumerate(ybins):
        yy = ((ybin-ymin)/yrange*yprange)+ypmin
        plt.plot([xpmin,xpmax],[yy,yy],color='grey',lw=0.2,zorder=1)
        if ibin % 2 == 0:
            plt.plot([xpmin-xprange*0.015,xpmin],[yy,yy],color='black',zorder=3)
            plt.annotate(np.round(ybin,1),(xpmin-xprange*0.08,yy),ha='center',size=tsize)
        else: 
            plt.plot([xpmin-xprange*0.05,xpmin],[yy,yy],color='black',zorder=3)
            plt.annotate(np.round(ybin,1),(xpmin-xprange*0.12,yy),ha='center',size=tsize)   
    
    XX = ((X-xmin)/xrange*xprange)+xpmin
    yy = ((y-ymin)/yrange*yprange)+ypmin
        
    plt.scatter(XX,yy,color='darkorange',edgecolor='black',s=30)
     
    plt.annotate(title,(xpmin+xprange*0.5,ypmax+yprange*0.08),ha='center') 
    plt.annotate(xlabel,(xpmin+xprange*0.5,ypmin-labelpad*1.5),ha='center') 
    plt.annotate(ylabel,(xpmin-labelpad*0.7,ypmin+yprange*0.5),va='center',rotation = 90.0) 
       
    y_input = -1
    if add_model == True:
        model = LinearRegression().fit(X.reshape(-1, 1),y)
        y_pred = model.predict(xbins.reshape(-1, 1))
        xxbins = ((xbins-xmin)/xrange*xprange)+xpmin
        yy_pred = ((y_pred-ymin)/yrange*yprange)+ypmin 
        plt.plot(xxbins,yy_pred,color='red',ls='--')
    
    if show_pred == True:
        xx_input = ((x_input-xmin)/xrange*xprange)+xpmin
        y_input = model.coef_[0]*x_input + model.intercept_
        yy_input = ((y_input-ymin)/yrange*yprange)+ypmin
        plt.plot([xx_input,xx_input],[ypmin,yy_input],color='black')
        plt.plot([xpmin,xx_input],[yy_input,yy_input],color='black')
        plt.annotate('f(' + str(np.round(x_input,1)) + ')=' + str(np.round(y_input,2)),(xpmin + 0.02*xprange,yy_input+0.05*yprange),size=tsize)
    return y_input     

#### Custom Bootstrap Visualization

We apply bootstrap data and statistic realizations to calculate the uncertainty in the average.

* In the display we make a simple data set for demonstration.
* Step through the bootstrap realizations or change the random number seed with the dashboard.

In [3]:
# parameters for the synthetic dataset
bins = np.linspace(0,1000,1000)

l = widgets.Text(value='                                   Boostrap Demonstration for Uncertainty in the Average, Michael Pyrcz, Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

nreal = widgets.IntSlider(min = 1, max = 16, value = 1, description = r'$L$',orientation='horizontal',layout=Layout(width='800px', height='20px'),continuous_update=False)
nreal.style.handle_color = 'gray'

seed = widgets.IntSlider(min = 1, max = 16, value = 1, description = r'$s$',orientation='horizontal',layout=Layout(width='800px', height='20px'),continuous_update=False)
seed.style.handle_color = 'gray'

ui5 = widgets.HBox([nreal,seed,],kwargs = {'justify_content':'center'})
ui_all = widgets.VBox([l,ui5],)

def make_viz3(nreal,seed):
    random.seed(seed); np.random.seed(seed=seed)
    random_values = np.random.normal(loc=10.0,scale=3.0,size=25)
    bootstrap_sample = np.zeros((nreal,len(random_values),))
    
    for ireal in range(0,nreal):
        bootstrap_sample[ireal] = random.choices(random_values, weights=None, cum_weights=None, k=len(random_values),)
    
    bootstrap_means = np.average(bootstrap_sample,axis = 1)    
    
    ax1 = plt.subplot(111)
    plt.axis('off'); 
    plt.xlim([0, 21]); plt.ylim([0, 30])
    xmin = 0; xmax = 20; ymin = 0; ymax = 10; nbin = 12; nybin = 5
    tsize = 6; labelpad = 1.2
    
    xorig = 1; yorig = 17
    xpmin = xorig; xpmax = xorig + 3.0; ypmin = yorig; ypmax =yorig + 4.0;
    custom_hist(random_values,ax1,xpmin,xpmax,ypmin,ypmax,xmin,xmax,ymin,ymax,nbin,nybin,'Porosity (%)','Frequency',
                'Original Sample Data',tsize,labelpad,plot_avg = False,plot_PDF = False)
        
    xorig = 5; yorig = 24
    lx = xorig; ly = yorig
    for ireal in range(0,nreal):
        xpmin = lx; xpmax = lx + 3.0; ypmin = ly; ypmax = ly + 4.0;
        custom_hist(bootstrap_sample[ireal],ax1,xpmin,xpmax,ypmin,ypmax,xmin,xmax,ymin,ymax,nbin,nybin,'Porosity (%)','Frequency',
                'Bootstrap Realization #' + str(ireal+1),tsize,labelpad,plot_avg = True,plot_PDF = False)
        lx = lx + 4
        if lx > 18:
            lx = xorig; ly = ly - 7
            
    xorig = 1; yorig = 9
    xpmin = xorig; xpmax = xorig + 3.0; ypmin = yorig; ypmax =yorig + 4.0; xmin = 8.0; xmax = 12.0
    if nreal < 5: 
        plot_PDF = False
    else: 
        plot_PDF = True
    custom_hist(bootstrap_means,ax1,xpmin,xpmax,ypmin,ypmax,xmin,xmax,ymin,ymax,int(nbin),nybin,'Porosity (%)','Frequency',
                'Bootstrap Averages',tsize,labelpad,plot_avg = False,plot_PDF = plot_PDF)
       
    plt.annotate('Bootstrap Uncertainty in Mean:',(0,5.8))
    plt.annotate('Mean: ' + str(np.round(np.average(bootstrap_means),2)),(1,4.5),ha='left')
    plt.annotate('St.Dev.: ' + str(np.round(np.std(bootstrap_means),2)),(0.8,3.5),ha='left')
    plt.annotate('P10: ' + str(np.round(np.percentile(bootstrap_means,10),2)),(1.2,2.5),ha='left')
    plt.annotate('P90: ' + str(np.round(np.percentile(bootstrap_means,90),2)),(1.2,1.5),ha='left')
        
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.2, hspace=0.2); 
    plt.savefig('Bootstrap.jpg',dpi=600,bbox_inches='tight')   
    plt.show()

interactive_plot1 = widgets.interactive_output(make_viz3, {'nreal':nreal,'seed':seed})
interactive_plot1.clear_output(wait = True)                # reduce flickering by delaying plot updating  

In [4]:
display(ui_all, interactive_plot1)                            # display the interactive plot

Output()

#### Custom Machine Learning Bagging Visualization

We apply bootstrap data and model prediction ensemble to demonstrate machine learning bagging.

* We make a simple data set for demonstration.
* Step through the bagging bootstrap data and model realizations or change the random number seed with the dashboard.
* Select a predictor feature value for the demonstrated prediction

In [13]:
seed = 13
np.random.seed(seed=seed)
num_samples = 30
mean = [0.0, 0.0]  # Mean vector
cov = [[1.0, 0.95], [0.95, 1.0]]  # Covariance matrix
X,y = np.random.multivariate_normal(mean, cov, num_samples).T
X = X * 0.2 + 2.2; y = y * 2.5 + 10.0

bins = np.linspace(0,1000,1000)
l = widgets.Text(value='        Demonstration of Machine Learning Ensemble Prediction with Bagging Linear Regression, Michael Pyrcz, Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))
nreal = widgets.IntSlider(min = 1, max = 16, value = 1, description = r'$L$',orientation='horizontal',layout=Layout(width='800px', height='20px'),continuous_update=False)
nreal.style.handle_color = 'gray'

x_input = widgets.FloatSlider(min = 1.8, max = 2.6, value = 2.3, description = r'$x$',orientation='horizontal',layout=Layout(width='800px', height='20px'),continuous_update=False)
x_input.style.handle_color = 'gray'

seed = widgets.IntSlider(min = 1, max = 16, value = 1, description = r'$s$',orientation='horizontal',layout=Layout(width='800px', height='20px'),continuous_update=False)
seed.style.handle_color = 'gray'

ui7 = widgets.HBox([nreal,x_input,seed,],kwargs = {'justify_content':'center'})
ui_all7 = widgets.VBox([l,ui7],)

def make_viz7(nreal,x_input,seed):
   
    ax1 = plt.subplot(111)
    
    xlabel = r'Density ($\frac{g}{cm^3}$)'; ylabel = 'Porosity (%)'; title = 'Original Sample Data'
    add_model = True; show_pred = True
    
    xorig = 1; yorig = 17; labelpad = 1.0
    xpmin = xorig; xpmax = xorig + 3.0; ypmin = yorig; ypmax =yorig + 3.0;
    random.seed(seed); np.random.seed(seed=seed)
    X_bootstrap = np.zeros((nreal,len(X))); y_bootstrap= np.zeros((nreal,len(X)))
    for ireal in range(0,nreal): # bootstrap from a bivariate dataset, sampling pairs
        index = random.choices(np.arange(0,len(X),1), weights=None, cum_weights=None, k=len(X),)
        X_bootstrap[ireal,:] = X[index]; y_bootstrap[ireal,:] = y[index]
        
    ax1 = plt.subplot(111)
    plt.axis('off'); 
    plt.xlim([0, 21]); plt.ylim([0, 30])
    xmin = 1.8; xmax = 2.6; ymin = 5; ymax = 15; nbin = 9; nybin = 5
    tsize = 6; labelpad = 1.2
    
    xorig = 1; yorig = 17
    xpmin = xorig; xpmax = xorig + 3.0; ypmin = yorig; ypmax = yorig + 5.0;
    
    custom_scatter(X,y,ax1,xpmin,xpmax,ypmin,ypmax,xmin,xmax,ymin,ymax,nbin,nybin,r'Density ($\frac{g}{cm^3}$)','Porosity (%)',
                    'Original Sample Data',tsize,labelpad,add_model = False,show_pred = False,x_input = -1.0)
    
    xorig = 5; yorig = 24
    lx = xorig; ly = yorig
    est_ensemble = np.zeros(nreal)
    for ireal in range(0,nreal):
        xpmin = lx; xpmax = lx + 3.0; ypmin = ly; ypmax = ly + 4.0;
        est_ensemble[ireal] = custom_scatter(X_bootstrap[ireal,:],y_bootstrap[ireal,:],ax1,xpmin,xpmax,ypmin,ypmax,
                xmin,xmax,ymin,ymax,nbin,nybin,r'Density ($\frac{g}{cm^3}$)','Porosity (%)',
                'Bootstrap Realization #' + str(ireal+1),tsize,labelpad,add_model,show_pred,x_input)
        lx = lx + 4
        if lx > 18:
            lx = xorig; ly = ly - 7
    
    xorig = 1; yorig = 9
    xpmin = xorig; xpmax = xorig + 3.0; ypmin = yorig; ypmax =yorig + 4.0
    xmin = 10.6; xmax = 12.0; ymin = 0; ymax = 10; nbin = 11
    if nreal < 5: 
        plot_PDF = False
    else: 
        plot_PDF = True 
    bag_est = np.average(est_ensemble)
    xbmin = np.round((bag_est - 0.5),1); xbmax = np.round((bag_est + 0.5),1);
    custom_hist(est_ensemble,ax1,xpmin,xpmax,ypmin,ypmax,xbmin,xbmax,ymin,ymax,nbin,nybin,r'Porosity Estimate (%)','Frequency',
            'Bagging Ensemble Estimate',tsize,labelpad,plot_avg = True,plot_PDF = plot_PDF)
            
    plt.annotate('Bagging Ensemble Predictions:',(0,5.8))
    plt.annotate('Mean: ' + str(np.round(np.average(est_ensemble),2)),(1,4.5),ha='left')
    plt.annotate('St.Dev.: ' + str(np.round(np.std(est_ensemble),2)),(0.8,3.5),ha='left')
    plt.annotate('P10: ' + str(np.round(np.percentile(est_ensemble,10),2)),(1.2,2.5),ha='left')
    plt.annotate('P90: ' + str(np.round(np.percentile(est_ensemble,90),2)),(1.2,1.5),ha='left')
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.2, hspace=0.2); 
    plt.savefig('Bootstrap.jpg',dpi=600,bbox_inches='tight')   
    plt.show() 
    
interactive_plot7 = widgets.interactive_output(make_viz7, {'nreal':nreal,'x_input':x_input,'seed':seed})
interactive_plot7.clear_output(wait = True)                # reduce flickering by delaying plot updating     

In [14]:
display(ui_all7, interactive_plot7)                            # display the interactive plot

Output()

#### Comments

This was a basic demonstration bootstrap and machine learning bagging with custom plots in an interactive Python dashboard. Much more can be done, I have other demonstrations for modeling workflows with GeostatsPy in the GitHub repository [GeostatsPy_Demos](https://github.com/GeostatsGuy/GeostatsPy_Demos/tree/main).

Note, the custom plots are rough, more could be done to format everything in a flexible manner, e.g., calculate the best label locations based on the size of the text in plot coordinates, etc.

I hope this is helpful,

*Michael*

#### The Author:

### Michael Pyrcz, Professor, The University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Professor, Cockrell School of Engineering and The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)  ://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Associate Professor The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)
